# 1、基础使用

举例

In [15]:
from langchain_chroma import Chroma
# 1.导入相关依赖
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
import dotenv

dotenv.load_dotenv()

# 2.定义文档加载器
loader = TextLoader(file_path='./asset/load/09-ai1.txt', encoding="utf-8")

# 3.加载文档
documents = loader.load()

# 4.定义文本切割器
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 5.切割文档
docs = text_splitter.split_documents(documents)

# 6.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)


In [26]:
from langchain_community.vectorstores import Chroma

print("type(docs[0]) ==>", type(docs[0]))
print("type(docs) ==>", type(docs))
print("type(embeddings) ==>", type(embeddings))

# 7.将文档存储到向量数据库中
db = Chroma.from_documents(docs, embeddings)

# 8.从向量数据库中得到检索器
retriever = db.as_retriever()

# 9.使用检索器检索
docs = retriever.invoke("深度学习是什么？")

print(len(docs))

# 10.得到结果
for doc in docs:
    print(f"⭐{doc}")

type(docs[0]) ==> <class 'langchain_core.documents.base.Document'>
type(docs) ==> <class 'list'>
type(embeddings) ==> <class 'langchain_openai.embeddings.base.OpenAIEmbeddings'>
4
⭐page_content='人工智能综述：发展、应用与未来展望

摘要
人工智能（Artificial Intelligence，AI）作为计算机科学的一个重要分支，近年来取得了突飞猛进的发展。本文综述了人工智能的发展历程、核心技术、应用领域以及未来发展趋势。通过对人工智能的定义、历史背景、主要技术（如机器学习、深度学习、自然语言处理等）的详细介绍，探讨了人工智能在医疗、金融、教育、交通等领域的应用，并分析了人工智能发展过程中面临的挑战与机遇。最后，本文对人工智能的未来发展进行了展望，提出了可能的突破方向。

1. 引言
人工智能是指通过计算机程序模拟人类智能的一门科学。自20世纪50年代诞生以来，人工智能经历了多次起伏，近年来随着计算能力的提升和大数据的普及，人工智能技术取得了显著的进展。人工智能的应用已经渗透到日常生活的方方面面，从智能手机的语音助手到自动驾驶汽车，从医疗诊断到金融分析，人工智能正在改变着人类社会的运行方式。

2. 人工智能的发展历程
2.1 早期发展
人工智能的概念最早可以追溯到20世纪50年代。1956年，达特茅斯会议（Dartmouth Conference）被认为是人工智能研究的正式开端。在随后的几十年里，人工智能研究经历了多次高潮与低谷。早期的研究主要集中在符号逻辑和专家系统上，但由于计算能力的限制和算法的不足，进展缓慢。
2.2 机器学习的兴起
20世纪90年代，随着统计学习方法的引入，机器学习逐渐成为人工智能研究的主流。支持向量机（SVM）、决策树、随机森林等算法在分类和回归任务中取得了良好的效果。这一时期，机器学习开始应用于数据挖掘、模式识别等领域。
2.3 深度学习的突破
2012年，深度学习在图像识别领域取得了突破性进展，标志着人工智能进入了一个新的阶段。深度学习通过多层神经网络模拟人脑的工作方式，能够自动提取特征并进行复杂的模式识别。卷积神经网络（CNN）、循环神经网

# 2、体会不同的检索策略

前置代码：

In [25]:
# 1.导入相关依赖
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# 2.定义文档
document_1 = Document(
    page_content="经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！",
)
document_2 = Document(
    page_content="基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。",
)
document_3 = Document(page_content="气候变化：承诺到2030年将温室气体排放量减少50%。",
                      )
document_4 = Document(
    page_content=" 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。",
)
document_5 = Document(
    page_content="教育：提供免费的社区大学教育。。",
)
document_6 = Document(
    page_content="科技：增加对半导体产业的投资以减少对外国供应链的依赖。。",
)
document_7 = Document(
    page_content="外交政策：继续支持乌克兰对抗俄罗斯的侵略。",
)
document_8 = Document(
    page_content="枪支管制：呼吁国会通过更严格的枪支管制法律。",
)
document_9 = Document(
    page_content="移民改革：提出全面的移民改革方案。",
)
document_10 = Document(
    page_content="社会正义：承诺解决系统性种族歧视问题。",
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

# 3.创建向量存储
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

# 4.将文档向量化，添加到向量数据库索引中，得到向量数据库对象
db = Chroma.from_documents(documents, embeddings)


① 默认检索器使用相似性搜索

In [27]:
# 获取检索器
retriever = db.as_retriever(search_kwargs={"k": 4})  #这里设置返回的文档数
docs = retriever.invoke("经济政策")
for i, doc in enumerate(docs):
    print(f"\n结果 {i + 1}:\n{doc.page_content}\n")


结果 1:
经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


结果 2:
科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


结果 3:
外交政策：继续支持乌克兰对抗俄罗斯的侵略。


结果 4:
基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。



② 分数阈值查询

只有相似度超过这个值才会召回


In [28]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)
docs = retriever.invoke("经济政策")
for doc in docs:
    print(f"📌 内容: {doc.page_content}")

📌 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


③ MMR搜索

In [29]:
retriever = db.as_retriever(
    search_type="mmr",
    # search_kwargs={"fetch_k":2}
)
docs = retriever.invoke("经济政策")
print(len(docs))
for doc in docs:
    print(f"📌 内容: {doc.page_content}")

4
📌 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！
📌 内容: 外交政策：继续支持乌克兰对抗俄罗斯的侵略。
📌 内容: 基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。
📌 内容: 社会正义：承诺解决系统性种族歧视问题。
